## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-09-18-52-31 +0000,wapo,Live updates: Israel and Hamas sign first phas...,https://www.washingtonpost.com/world/2025/10/0...
1,2025-10-09-18-52-31 +0000,wapo,Hamas received guarantee war ‘fully and comple...,https://www.washingtonpost.com
2,2025-10-09-18-50-01 +0000,nyt,Live Updates: Trump Calls Gaza Deal a ‘Momento...,https://www.nytimes.com/live/2025/10/09/world/...
3,2025-10-09-18-50-01 +0000,nyt,Trump Baselessly Claims He ‘Took the Freedom o...,https://www.nytimes.com/2025/10/09/us/politics...
4,2025-10-09-18-49-43 +0000,nypost,"Circumcision ‘highly likely’ linked to autism,...",https://nypost.com/2025/10/09/us-news/tylenol-...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2303/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
20,trump,43
9,deal,28
7,gaza,27
41,new,22
3,hamas,22


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
261,2025-10-08-23-17-50 +0000,nypost,Trump announces Israel and Hamas signed off on...,https://nypost.com/2025/10/08/world-news/trump...,192
0,2025-10-09-18-52-31 +0000,wapo,Live updates: Israel and Hamas sign first phas...,https://www.washingtonpost.com/world/2025/10/0...,144
279,2025-10-08-22-30-24 +0000,wapo,"Trump, Hamas officials say Gaza ceasefire deal...",https://www.washingtonpost.com/world/2025/10/0...,135
152,2025-10-09-11-51-45 +0000,nypost,Gazans join Israelis in chanting Trump’s name ...,https://nypost.com/2025/10/09/us-news/gazans-j...,131
221,2025-10-09-04-23-07 +0000,nyt,President Trump Says Israel and Hamas Agree to...,https://www.nytimes.com/2025/10/08/world/middl...,131


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
261,192,2025-10-08-23-17-50 +0000,nypost,Trump announces Israel and Hamas signed off on...,https://nypost.com/2025/10/08/world-news/trump...
264,73,2025-10-08-23-12-49 +0000,latimes,"After Palisades fire failures, L.A. promises t...",https://www.latimes.com/california/story/2025-...
170,65,2025-10-09-10-39-00 +0000,wsj,China introduced new controls on rare-earth ex...,https://www.wsj.com/economy/trade/china-impose...
253,44,2025-10-08-23-35-52 +0000,nypost,Ciattarelli gives President Trump an ‘A’ grade...,https://nypost.com/2025/10/08/us-news/ciattare...
294,42,2025-10-08-21-33-46 +0000,nypost,Andrew Cuomo would scrap plan to close Rikers ...,https://nypost.com/2025/10/08/us-news/andrew-c...
55,40,2025-10-09-16-51-13 +0000,nypost,California becomes first state to phase out ul...,https://nypost.com/2025/10/09/us-news/californ...
246,38,2025-10-09-00-19-19 +0000,nypost,Watch viral shouting match between Mike Lawler...,https://nypost.com/2025/10/08/us-news/mike-law...
307,37,2025-10-08-20-56-43 +0000,nypost,Supreme Court likely to favor GOP challenge to...,https://nypost.com/2025/10/08/us-news/supreme-...
33,37,2025-10-09-17-37-51 +0000,nypost,Portland police union head welcomes fed suppor...,https://nypost.com/2025/10/09/us-news/portland...
256,33,2025-10-08-23-25-18 +0000,nyt,"Texts from Jay Jones, Democratic Attorney Gene...",https://www.nytimes.com/2025/10/08/us/jay-jone...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
